In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import scipy as sc
from scipy import signal
import cv2 
import PIL

In [2]:
df = pd.read_csv(r"E:\Academic\Project\User Identity from Time-Frequency Analysis\50_Users_Acceleromter_Data.csv")
df

,X,Y,Z,Label
0,0.340509,8.308413,4.140585,0
1,0.381370,8.390134,4.249548,0
2,0.272407,8.471856,4.018002,0
3,0.149824,8.430995,4.290409,0
4,0.272407,8.430995,4.481094,0
...,...,...,...,...
749995,-7.201759,-4.750096,-1.532289,49
749996,-7.354988,-5.209783,-1.991976,49
749997,-7.201759,-5.056554,-2.145205,49
749998,-7.354988,-5.363012,-2.298433,49


In [3]:
length = 15000

In [4]:
import time
start_time = time.time()
N_Users=50
k=0
X=[0 for p in range(N_Users)] # Final Array
for i in range(0,length*N_Users,length):
    temp=[]
    for j in range(length):
        x_y_z = [df.iloc[i+j,0],df.iloc[i+j,1],df.iloc[i+j,2]]
        temp.append(x_y_z)
    X[k]=temp
    k+=1 #incrementing [Final Array] indices
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.418941259384155 seconds ---


In [5]:
t=np.array(X)
t.shape

(50, 15000, 3)

In [6]:
# Main Algo for Creating Windows .....#1

def create(df,wind_size,hop_size,m_index):
  #l=((len(df.x)-wind_size)/hop_size)+1
  rows, cols = (wind_size, 3) 
#  a = [[0 for i in range(cols)] for j in range(rows)] 
  array= [0 for i in range(int((length-wind_size)/hop_size)+1)]
  k = 0
  for i in range(0,(length-wind_size+1),hop_size):
      a=create1(df,i,wind_size,m_index)
       #array.append(a)
      array[k] = a
      k+=1
  #return pd.DataFrame(array)
  return array


In [8]:
# Main Algo for Creating Windows .....#2

def create1(df,x,w,m_index):
     rows = w 
     cols = 3
     arr = [[0 for i in range(cols)] for j in range(rows)] 
     for i in range(0,w,1):
        arr[i][0]=df[x+i][0]
        arr[i][1]=df[x+i][1]
        arr[i][2]=df[x+i][2]
     arr = np.array(arr)
     f1,t1,x = signal.stft(arr[:,0],5,nperseg=10)
     f2,t2,y = signal.stft(arr[:,1],5,nperseg=10)
     f3,t3,z = signal.stft(arr[:,2],5,nperseg=10)
     temp_a = np.concatenate((abs(np.array(x)),abs(np.array(y)),abs(np.array(z))),axis=0) 
     return temp_a

In [9]:
import time
start_time = time.time()
X_final = [0 for i in range( N_Users)]
for m_index in range(N_Users):
  X_final[m_index]=create(X[m_index],225,10,m_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 59.8059196472168 seconds ---


In [10]:
t=np.array(X_final[0])
t.shape             # The shape gives(number of windows,shape of TF plot image)

(1478, 18, 46)

In [11]:
X_input=[]
for i in range(N_Users):
  temp = X_final[i]
  X_input += temp
X_input = np.array(X_input)

In [12]:
Address_list=[]
for k in range(N_Users):
    for _ in range (t.shape[0]):
        Address_list.append(str(r'E:\Academic\Project\User Identity from Time-Frequency Analysis\Data\User_'+str(k+1)+'\\'+str(_+1)+".png"))
    

In [16]:
Address_list[0]

'E:\\Academic\\Project\\User Identity from Time-Frequency Analysis\\Data\\User_1\\1.png'

In [36]:
import skimage
from skimage.io import imsave
for r in range(len(Address_list)):
    data = X_input[r].astype(np.float64)/np.max(X_input[r]) # normalize the data to 0 - 1
    data = 255 * data # Now scale by 255
    img = data.astype(np.uint8)
    imsave(Address_list[r],img)